# Background

The following ABM was created as practice for my coding in Mesa. The situation is one I've personally imagined involving the emerald ash borer (EAB), an invasive wood-boring beetle from Asia responsible for the death and decline of millions of ash trees in North America. The emerald ash borer lays its eggs in the bark crevices of these ash trees, and the larvae from these eggs burrow into the tree where they feed. The purpose of the ABM is to examine how pest control methods, specifically the spacing of vulnerable trees, can help mitigate the pest problem. Some assumptions are made to simplify the model.

# Import Dependencies

In [89]:
import mesa
import random
import numpy as np
import matplotlib.pyplot as plt
import warnings

from mesa.experimental import JupyterViz

In [90]:
# To get rid of UserWarning that keeps popping up for some reason regarding placing the same agent in a different position 
warnings.simplefilter("ignore", UserWarning)

# Agent Classes

In [92]:
class Ash_Tree(mesa.Agent):
    def __init__(self, unique_id, model, pos, condition):
        """
        Initializer function for ash trees' id, position, and health condition
        """
        super().__init__(unique_id, model)
        self.pos = pos
        self.condition = condition 
    
    def maybe_spread_infection(self):
        """
        Function called in step() where ash trees that are infected with EAB spread their infection to neighboring trees
        """
        if self.condition == "Infected":
            neighbors = self.model.grid.get_neighbors(self.pos, moore=True, include_center=False)
            for neighbor in neighbors:
                if isinstance(neighbor, Ash_Tree) and neighbor.condition == "Healthy":
                    neighbor.condition = "Infected"
            self.condition = "Dead"

    def step(self):
        """
        Step function which occurs with each step of the model for each ash tree in the model
        """
        self.maybe_spread_infection()

# Model Class

In [94]:
class Forest(mesa.Model):
    def __init__(self, width, height, tree_density):
        """
        Initializer function which creates the ash trees and adds them to the grid and scheduler
        """
        super().__init__()
        self.width = width
        self.height = height
        self.tree_density = tree_density
        self.running = True
        
        # Initialize scheduler
        self.schedule = mesa.time.RandomActivation(self)
        # Initialize grid class
        self.grid = mesa.space.SingleGrid(self.width, self.height, torus=False)
        # Initialize data collector 
        self.datacollector = mesa.DataCollector(
            {
                "Healthy": lambda m: self.count_type(m, "Healthy"),
                "Infected": lambda m: self.count_type(m, "Infected"),
                "Dead": lambda m: self.count_type(m, "Dead"),
            }
        )

        # Create a list of all possible positions
        all_possible_positions = [(x, y) for x in range(self.width) for y in range(self.height)]
        # Shuffle the positions
        self.random.shuffle(all_possible_positions)
        # Calculate the number of trees to create so that we can place an ash tree with probability given by the model 
        num_of_agents = int(self.width*self.height*self.tree_density)

        # Create ash trees and add them to the scheduler and grid        
        for i in range(num_of_agents):
            x,y = all_possible_positions[i]
            if x == 0: 
                condition = "Infected"
            else: 
                condition = "Healthy"
            new_tree = Ash_Tree(i, self, (x,y), condition)
            self.grid.place_agent(new_tree, (x,y))
            self.schedule.add(new_tree)
                                    
    def step(self):
        """
        Step function that advances the model 
        """
        self.schedule.step()
        self.datacollector.collect(self)
        if self.count_type(self, "Infected") == 0:
            self.running = False

    @staticmethod
    def count_type(model, tree_condition):
        """
        Helper method to count trees in a given condition in a given model.
        """
        count = 0
        for tree in model.schedule.agents:
            if tree.condition == tree_condition:
                count += 1
        return count

# Running the Model

In [96]:
def run_model(width, height, tree_density, max_steps):
    model = Forest(width, height, tree_density)
    for i in range(max_steps):
        if not model.running:
            break
        model.step()
    return model.datacollector.get_model_vars_dataframe()

# Getting Results

In [98]:
results = run_model(50, 50, 0.3, 100)
print(results)

plt.figure(figsize=(10, 6))
plt.plot(results["Infected"], label="Infected")
plt.plot(results["Healthy"], label="Healthy")
plt.plot(results["Dead"], label="Dead")
plt.xlabel("Steps")
plt.ylabel("Number of Trees")
plt.title("Spread of Infection in Forest")
plt.legend()
plt.show()

   Healthy  Infected  Dead
0      731         4    15
1      724         3    23
2      715         4    31
3      714         1    35
4      713         1    36
5      713         0    37


In [99]:
results = run_model(50, 50, 0.7, 100)
print(results)

plt.figure(figsize=(10, 6))
plt.plot(results["Infected"], label="Infected")
plt.plot(results["Healthy"], label="Healthy")
plt.plot(results["Dead"], label="Dead")
plt.xlabel("Steps")
plt.ylabel("Number of Trees")
plt.title("Spread of Infection in Forest")
plt.legend()
plt.show()

    Healthy  Infected  Dead
0      1620        53    77
1      1518        55   177
2      1379        63   308
3      1245        78   427
4      1125        59   566
5      1022        54   674
6       902        76   772
7       770        70   910
8       628        75  1047
9       487        80  1183
10      362        80  1308
11      253        58  1439
12      143        58  1549
13       64        36  1650
14       21        19  1710
15        3         8  1739
16        2         1  1747
17        2         0  1748


# Visualization 

In [101]:
model_params = {
    "tree_density": {
        "type": "SliderFloat",
        "value": 0.4,
        "label": "Tree Density:",
        "min": 0,
        "max": 1,
        "step": 0.05,
    },
    "width": 50,
    "height": 50,
}

def tree_portrayal(ash_tree):
    size = 20
    if ash_tree.condition == "Healthy":
        color = "tab:green"
    elif ash_tree.condition == "Infected":
        color = "tab:red"
    elif ash_tree.condition == "Dead":
        color = "tab:blue"
    return {"size": size, "color": color}

page = JupyterViz(
    Forest,
    model_params,
    measures=["Healthy"],
    name="EAB Infection Model",
    agent_portrayal=tree_portrayal,
)

page


Cannot show ipywidgets in text